In [4]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
from tqdm import tqdm

In [5]:
# Hyperparameters
player_names = ["Thomas", "Emily", "Benjamin", "Sophia", "Victoria"]
roles = [0,0,0,1,1]

model = "gpt-3.5-turbo-16k" # "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf"
}

In [6]:
n_games = 20

for game in tqdm(range(2, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Init all the players
    players : Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=["The werewolfs (including you) are [" + ",".join(werewolf_names) + "]."] if name.lower() in werewolf_names else [])
        players[name.lower()].init_player(players=player_names)

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []

    # Debate stage
    while rounds < 15 and not stop:
        rounds += 1
        next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-4-0314").lower()

        if next_to_speak != "vote":    
            censored, uncensored = players[next_to_speak].get_player_text()
            
            for name in player_names:
                if name.lower() != next_to_speak:
                    players[name.lower()].add_other_text(censored)
        
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)

            print(uncensored)
        else:
            break

    # Vote stage
    votes = {name.lower(): 0 for name in player_names}

    vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
    conversation_history.append(vote_prompt.content)

    print(vote_prompt.content)

    for name in player_names:
        players[name.lower()].add_other_text(vote_prompt.content)
        if name.lower() in werewolf_names:
            players[name.lower()].current_context.content += f"\n{name}: [I know that I am the werewolf. The werewolf are (including me): {', '.join(werewolf_names)}, I must not vote for any werewolf]"

    for voting_name in player_names:
        censored, uncensored = players[voting_name.lower()].get_player_text()

        for name in player_names:
            if name.lower() != voting_name:
                players[name.lower()].add_other_text(censored)
        
        conversation_history.append(censored)
        conversation_history_uncensored.append(uncensored)

        voted_for = ""
        search = censored.lower()
        for name in player_names:
            if name.lower() in search:
                voted_for = name.lower()
                search = search.split(name.lower())[-1]
    
        try:
            votes[voted_for] += 1
        except:
            pass

        print(uncensored)
    print(votes)

    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "player_names": player_names,
        "model": model
    }, open(f"../data/experiment_gpt3.5-turbo-16k/game_number_{game}.json", "w+"), indent=4)

Thomas: Good evening, fellow villagers. It deeply saddens me to inform you that our dear friend Bob has been taken from our midst by the treacherous claws of the Werewolf. We must act swiftly and decisively to protect our village. I urge each of you to share any suspicions or observations you may have. We cannot afford to be complacent in the face of this grave threat.


In [ ]:
print(votes)